# 1.Import packages

In [1]:
# Importing all required packages at the start of the notebook
import IPython

from qiime2 import Visualization

import qiime2 as q2
import pandas as pd
import matplotlib.pyplot as plt
import os

%matplotlib inline

# 2.Import the data

In [2]:
# Location of the projects data
!mkdir -p "Project_data"
data_dir = "Project_data/Import_and_Denoizing"

In [3]:
%%bash -s $data_dir
mkdir -p "$1"

wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza

chmod -R +rxw "$1"

--2025-11-03 15:19:40--  https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712595535 (680M) [application/octet-stream]
Saving to: ‘Project_data/Import_and_Denoizing/fungut_forward_reads.qza’

     0K ........ ........ ........ ........  4%  132M 5s
 32768K ........ ........ ........ ........  9%  128M 5s
 65536K ........ ........ ........ ........ 14%  126M 5s
 98304K ........ ........ ........ ........ 18%  130M 4s
131072K ........ ........ ........ ........ 23%  114M 4s
163840K ........ ........ ........ ........ 28%  136M 4s
196608K ........ ........ ........ ........ 32%  133M 4s
229376K ........ ........ ........ ........ 37%  114M 3s
262144K ........ ........ ........ ........ 42%  147M 3s
294912K ........ ........ ........ ........ 47%  143M 3s
327

# 3.Feature table construction

In [4]:
# Visual summary of the data
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads.qza \
    --o-visualization $data_dir/fungut_forward_reads_demux_seqs.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/fungut_forward_reads_demux_seqs.qzv


In [3]:
Visualization.load(f"{data_dir}/fungut_forward_reads_demux_seqs.qzv")

<visualization: Visualization uuid: b5307e0f-14b1-47f6-b251-c3ef573d4d2e>

Seeing that the quality of the reads stays quite high even at the last position (151 nts), we first tried truncating at this length. However, this led to a major loss of sequences when comparing denoising steps with a truncating length of 130 or 140. We also tried another denoizing technique, with no truncation at all, filtering afterhand the sequences that were present in too little samples and the ones present at a too little frequence, but that also led to the loss of a lot of sequencing. Finally, we settled down for a length of truncation of 135, because it gave us a good output when looking at the sequencing stats.

# Denoising - Amplicon Sequence Variants

In [4]:
! qiime quality-filter q-score \
    --i-demux $data_dir/fungut_forward_reads.qza \
    --p-min-quality 30 \
    --o-filtered-sequences $data_dir/fungut_forward_reads_quality_filtered.qza \
    --o-filter-stats $data_dir/quality_filtering_stats.qza 

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved SampleData[SequencesWithQuality] to: Project_data/Import_and_Denoizing/fungut_forward_reads_quality_filtered.qza
Saved QualityFilterStats to: Project_data/Import_and_Denoizing/quality_filtering_stats.qza


In [5]:
! qiime metadata tabulate \
    --m-input-file $data_dir/quality_filtering_stats.qza \
    --o-visualization $data_dir/quality_filtering_stats.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/quality_filtering_stats.qzv


In [6]:
Visualization.load(f"{data_dir}/quality_filtering_stats.qzv")

<visualization: Visualization uuid: d6d7f5e7-a0d2-4bc4-8dc3-2d0c864d1d9b>

In [7]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir/fungut_forward_reads_quality_filtered.qza \
    --p-trunc-len 0 \
    --p-n-threads 3 \
    --o-table $data_dir/dada2_table.qza \
    --o-representative-sequences $data_dir/dada2_rep_set.qza \
    --o-denoising-stats $data_dir/dada2_stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: Project_data/Import_and_Denoizing/dada2_table.qza
Saved FeatureData[Sequence] to: Project_data/Import_and_Denoizing/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: Project_data/Import_and_Denoizing/dada2_stats.qza


In [8]:
! qiime metadata tabulate \
  --m-input-file $data_dir/dada2_stats.qza \
  --o-visualization $data_dir/dada2_stats.qzv

! qiime feature-table tabulate-seqs \
  --i-data $data_dir/dada2_rep_set.qza \
  --o-visualization $data_dir/dada2_rep_set.qzv

! qiime feature-table summarize \
  --i-table $data_dir/dada2_table.qza \
  --m-sample-metadata-file Project_data/Metadata/updated_fungut_metadata.tsv \
  --o-visualization $data_dir/dada2_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/dada2_stats.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Import_and_Denoizing/dada2_rep_set.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources 

In [3]:
Visualization.load(f"{data_dir}/dada2_stats.qzv")

<visualization: Visualization uuid: c9126808-c678-4a86-9add-00b50ea4cdee>

In [4]:
Visualization.load(f"{data_dir}/dada2_rep_set.qzv")

<visualization: Visualization uuid: c676df41-df90-489b-917b-1c16c468766e>

In [5]:
Visualization.load(f"{data_dir}/dada2_table.qzv")

<visualization: Visualization uuid: 690b85d4-5c47-437a-af92-f879e941c561>